In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
from keras.models import Model
from keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Flatten, Dense #Conv2D, MaxPooling2D, 
from tensorflow.keras.optimizers import SGD
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
import os, sys

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
train_set_path = '/content/drive/MyDrive/Colab Notebooks/cats_and_dogs/train'
test_set_path = '/content/drive/MyDrive/Colab Notebooks/cats_and_dogs/test'

In [5]:
# Use prebuild cnn model VGG16 
# Input shape = (224, 224)
def define_model():
  	# load model
	model = VGG16(include_top=False, input_shape=(224, 224, 3))
	# mark loaded layers as not trainable
	for layer in model.layers:
		layer.trainable = False
	# add new classifier layers
	flat1 = Flatten()(model.layers[-1].output)
	class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
	output = Dense(1, activation='sigmoid')(class1)
	# define new model
	model = Model(inputs=model.inputs, outputs=output)
	# compile model
	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
	return model

In [6]:
# plot diagnostic learning curves
def summarize_diagnostics(history):
  # plot loss
  plt.subplot(211)
  plt.title('Cross Entropy Loss')
  plt.plot(history.history['loss'], color='blue', label='train')
  plt.plot(history.history['val_loss'], color='orange', label='test')
  # plot accuracy
  plt.subplot(212)
  plt.title('Classification Accuracy')
  plt.plot(history.history['accuracy'], color='blue', label='train')
  plt.plot(history.history['val_accuracy'], color='orange', label='test')
  # save plot to file
  #filename = sys.argv[0].split('/')[-1]
  #plt.savefig(filename + '_plot.png')
  plt.show()
  plt.close()

In [7]:
# Data generator, same generator for train set and test set
data_gen = ImageDataGenerator(rescale=1.0/255.0)

train_gen = data_gen.flow_from_directory(train_set_path,
                                        class_mode='binary',
                                        batch_size=64,
                                        target_size=(224, 224))

test_gen = data_gen.flow_from_directory(test_set_path,
                                        class_mode='binary',
                                        batch_size=64,
                                        target_size=(224, 224))

Found 18697 images belonging to 2 classes.
Found 6301 images belonging to 2 classes.


In [ ]:
# Train a model
model = define_model()

monitor = EarlyStopping(monitor='val_loss', 
                        min_delta=1e-3, 
                        patience=5, 
                        verbose=1, 
                        mode='auto',
                        restore_best_weights=True)

history = model.fit(train_gen, 
                    steps_per_epoch=len(train_gen),
                    validation_data=test_gen, 
                    validation_steps=len(test_gen), 
                    epochs=100, 
                    verbose=1,
                    callbacks=[monitor])

summarize_diagnostics(history)

58900480/58889256 [==============================] - 1s 0us/step


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/100
 53/293 [====>.........................] - ETA: 2:22:42 - loss: 0.4857 - accuracy: 0.7574

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 270
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5 bytes but only got 0. Skipping tag 271
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 272
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only

293/293 [==============================] - 13561s 46s/step - loss: 0.3120 - accuracy: 0.8584 - val_loss: 0.2418 - val_accuracy: 0.8932
Epoch 2/100
293/293 [==============================] - ETA: 0s - loss: 0.2036 - accuracy: 0.9149 